In [1]:
from ultralytics import YOLO
import cv2
import json
import os

In [2]:
# Load the YOLOv8 pre-trained model
model = YOLO("yolov8n.pt")  # Use 'yolov8n.pt', 'yolov8s.pt', etc., for different sizes


In [3]:
# Define COCO class IDs for vehicles
vehicle_classes = [2, 3, 5, 7]  # Cars, Motorcycles, Buses, Trucks

# Initialize storage for predictions
predictions_file = "traffic_predictions.json"

In [4]:
# Function to load previous predictions
def load_previous_predictions():
    if os.path.exists(predictions_file):
        with open(predictions_file, "r") as file:
            return json.load(file)
    return {"prev_vehicle_count": 0, "prev_red_time": 0}

# Function to save predictions
def save_predictions(predictions):
    with open(predictions_file, "w") as file:
        json.dump(predictions, file)


In [5]:
# Function to process an image and make predictions
def process_image(image_path, output_path="output_image.jpg"):
    # Load previous predictions
    prev_data = load_previous_predictions()
    prev_vehicle_count = prev_data["prev_vehicle_count"]
    prev_red_time = prev_data["prev_red_time"]

    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Cannot read the image file.")
        return

    # Run YOLOv8 inference on the image
    results = model(image)

    # Extract bounding boxes and class IDs
    vehicle_count = 0
    for result in results[0].boxes.data.tolist():
        x1, y1, x2, y2, conf, class_id = result
        class_id = int(class_id)
        if class_id in vehicle_classes:  # Check if it's a vehicle
            vehicle_count += 1
            # Draw bounding box
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            # Add label
            cv2.putText(
                image,
                model.names[class_id],
                (int(x1), int(y1) - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                2,
            )

    # Calculate green and red light durations
    green_light_duration = vehicle_count * 2  # Example: 2 seconds per vehicle
    red_light_duration = prev_vehicle_count * 1.5  # Example: 1.5 seconds per vehicle

    # Display the predictions on the image
    cv2.putText(
        image,
        f"Vehicle Count: {vehicle_count}",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
    )
    cv2.putText(
        image,
        f"Green Light: {green_light_duration}s",
        (10, 60),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
    )
    cv2.putText(
        image,
        f"Red Light: {red_light_duration:.1f}s",
        (10, 90),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (255, 0, 0),
        2,
    )

    # Save the output image
    cv2.imwrite(output_path, image)
    print(f"Output saved at: {output_path}")

    # Save the predictions for the next iteration
    predictions = {
        "prev_vehicle_count": vehicle_count,
        "prev_red_time": red_light_duration,
    }
    save_predictions(predictions)

    # Display the image (optional)
    cv2.imshow("Vehicle Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
# Run the function on an image
process_image("traffic1.jpg", "output_with_predictions.jpg")


0: 384x640 16 cars, 2 buss, 1 truck, 173.5ms
Speed: 10.8ms preprocess, 173.5ms inference, 13.2ms postprocess per image at shape (1, 3, 384, 640)
Output saved at: output_with_predictions.jpg
